In [232]:
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from copy import deepcopy
import data_generation as dg

In [233]:
graph, random_solution, total_edge_length = dg.read_instance('data\data20.txt')

In [234]:
def make_change_swap(graph, solution):
    new_solution = solution.copy()
    
    # Choose two distinct random indices
    index1, index2 = random.sample(range(len(solution)), 2)
    
    # Swap the positions of the selected nodes
    new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]

    return new_solution

In [235]:
def make_change_inverse(graph, solution):
    new_solution = solution.copy()
    
    # Choose two distinct random indices
    index1, index2 = sorted(random.sample(range(len(solution)), 2))
    
    # Reverse the subset of nodes between index1 and index2
    new_solution[index1:index2+1] = reversed(new_solution[index1:index2+1])

    return new_solution

In [236]:
def make_change_next_permutation(graph, solution):
    #based on the classic next permutation algorithm
    new_solution = deepcopy(solution)
    
    n = len(new_solution)
    
    # Find the largest index k such that a[k] < a[k+1]
    k = n - 2
    while k >= 0 and new_solution[k] >= new_solution[k + 1]:
        k -= 1

    # If no such index exists, the permutation is the last one
    if k == -1:
        return sorted(new_solution)

    # Find the largest index l greater than k such that a[k] < a[l]
    l = n - 1
    while new_solution[k] >= new_solution[l]:
        l -= 1

    # Swap a[k] and a[l]
    new_solution[k], new_solution[l] = new_solution[l], new_solution[k]

    # Reverse the sequence from a[k+1] up to and including the final element a[n-1]
    new_solution[k + 1:] = reversed(new_solution[k + 1:])

    return new_solution


In [237]:
def make_change_scramble(graph, solution):
    new_solution = deepcopy(solution)
    
    start, end = sorted(random.sample(range(len(solution)), 2))

    subset = solution[start:end+1]

    random.shuffle(subset)

    new_solution[start:end+1] = subset

    return new_solution

In [238]:
def local_search(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(num_iters):
        #print(solution, value, i)
        new_solution = change_func(graph, solution)
        new_value = dg.calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

    return best_solution, best_value, best_i

In [239]:
local_search(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)

([2, 1, 16, 13, 17, 8, 7, 6, 4, 11, 12, 9, 14, 18, 3, 10, 19, 5, 0, 15],
 429,
 499)

In [240]:
local_search(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)

([8, 1, 2, 16, 11, 13, 18, 4, 6, 17, 12, 9, 14, 7, 5, 3, 19, 0, 10, 15],
 417,
 493)

In [241]:
local_search(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble)

([13, 16, 17, 7, 4, 12, 3, 6, 14, 10, 9, 2, 0, 5, 19, 18, 11, 1, 8, 15],
 431,
 439)

In [242]:
def local_search_with_permutation(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(num_iters):
        #print(solution, value, i)
        new_solution = change_func(graph, solution)
        new_value = dg.calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

        else:
            perm_counter = 0
            perm_limit = num_iters / (i+1)
            while new_value >= value and perm_counter < perm_limit:    
                new_solution = make_change_next_permutation(graph, new_solution)
                new_value = dg.calculate_total_edge_length(graph, new_solution)
                perm_counter += 1

            if new_value < value:
                value = new_value
                solution = deepcopy(new_solution)

                if new_value < best_value:
                    best_i = i
                    best_value = new_value
                    best_solution = deepcopy(new_solution)
              

    return best_solution, best_value, best_i

In [243]:
local_search_with_permutation(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)

([15, 10, 0, 19, 14, 5, 2, 3, 18, 12, 9, 4, 7, 17, 6, 16, 13, 11, 1, 8],
 418,
 334)

In [244]:
local_search_with_permutation(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)

([1, 16, 2, 8, 13, 17, 11, 6, 9, 7, 4, 14, 12, 18, 3, 5, 19, 0, 10, 15],
 421,
 426)

In [245]:
local_search_with_permutation(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble)

([8, 1, 2, 16, 11, 18, 9, 19, 5, 14, 12, 17, 3, 6, 0, 4, 10, 13, 7, 15],
 410,
 474)

In [246]:
def simulated_annealing(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(1, num_iters + 1):
        #print(solution, value)
        new_solution = change_func(graph, solution)
        new_value = dg.calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

        #elif random.random() < 1 / (i**0.5):
        elif random.random() < 1 / i:
            #print('divs')
            value = new_value
            solution = deepcopy(new_solution)

    return best_solution, best_value, best_i

In [247]:
simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)

([8, 1, 2, 11, 18, 19, 5, 14, 9, 0, 12, 3, 6, 17, 16, 4, 7, 13, 10, 15],
 411,
 176)

In [248]:
simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)

([15, 13, 7, 16, 17, 4, 10, 3, 12, 14, 6, 9, 5, 19, 11, 0, 18, 2, 1, 8],
 419,
 415)

In [249]:
simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble)

([1, 8, 15, 11, 13, 7, 6, 4, 17, 9, 2, 16, 12, 18, 3, 14, 0, 10, 19, 5],
 432,
 498)

In [250]:
def shaking_swap(graph, solution, k):
    new_solution = deepcopy(solution)
    selected = random.sample(range(len(solution)), 2*k)
    for i in range(0,len(selected), 2):
        index1, index2 = selected[i], selected[i+1]
        new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]
    return new_solution    
    

In [251]:
def shaking_inverse(graph, solution, k):
    new_solution = deepcopy(solution)
    index1 = random.randint(0, len(solution))
    if(index1 + k > len(solution)):
        index1 = len(solution) - k
    index2 = index1 + k                       
    new_solution[index1:index2+1] = reversed(new_solution[index1:index2+1])  

    return new_solution

In [252]:
def vns(graph, random_solution, value, num_iters, change_func, shaking_func, local_search_func, k_min, k_max, move_prob):
    solution = deepcopy(random_solution)
    best_i = None
    for i in range(num_iters):
        for k in range(k_min, k_max):
            #print('vns: ', solution, value, i)
            new_solution = shaking_func(graph, solution, k) #diversification  

            new_value = dg.calculate_total_edge_length(graph, new_solution)

            #print('post shaking: ', new_solution, new_value, i)
            new_solution, new_value, _= local_search_func(graph, new_solution, total_edge_length, 10, change_func)

            if new_value < value or (new_value == value and random.random() < move_prob):
                if(new_value < value):
                    best_i = i

                value = new_value

                solution = deepcopy(new_solution)
                
    return solution, value, best_i

In [253]:
vns(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse, shaking_func=shaking_swap, local_search_func = local_search, k_min=1, k_max=3, move_prob=0.4)

([8, 1, 11, 2, 18, 19, 5, 9, 14, 16, 12, 3, 6, 17, 4, 0, 10, 13, 7, 15],
 409,
 350)

In [254]:
vns(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse, shaking_func=shaking_inverse, local_search_func = local_search, k_min=3, k_max=6, move_prob=0.4)

([15, 10, 0, 19, 5, 3, 14, 18, 12, 6, 9, 7, 17, 4, 13, 11, 16, 2, 8, 1],
 412,
 452)

In [255]:
vns(graph, random_solution, total_edge_length, 500, change_func=make_change_swap, shaking_func=shaking_swap, local_search_func = local_search, k_min=1, k_max=3, move_prob=0.4)

([15, 13, 7, 10, 0, 4, 6, 17, 3, 16, 12, 14, 9, 19, 5, 11, 18, 2, 8, 1],
 410,
 240)

In [256]:
vns(graph, random_solution, total_edge_length, 500, change_func=make_change_swap, shaking_func=shaking_inverse, local_search_func = local_search, k_min=3, k_max=6, move_prob=0.4)

([1, 8, 2, 11, 18, 14, 19, 9, 5, 12, 17, 16, 6, 3, 7, 4, 13, 0, 10, 15],
 415,
 176)

In [257]:
vns(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble, shaking_func=shaking_swap, local_search_func = local_search, k_min=1, k_max=3, move_prob=0.4)

([15, 0, 10, 19, 5, 3, 14, 12, 18, 7, 4, 6, 9, 17, 13, 2, 16, 11, 1, 8],
 411,
 355)

In [258]:
vns(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble, shaking_func=shaking_inverse, local_search_func = local_search, k_min=3, k_max=6, move_prob=0.4)

([8, 1, 11, 13, 16, 6, 4, 17, 9, 7, 12, 2, 18, 14, 3, 5, 19, 0, 10, 15],
 411,
 423)

In [259]:
vns(graph, random_solution, total_edge_length, 500, change_func=make_change_swap, shaking_func=shaking_swap, local_search_func = local_search_with_permutation, k_min=1, k_max=3, move_prob=0.4)

([15, 7, 13, 10, 4, 0, 3, 6, 17, 16, 12, 14, 9, 19, 2, 5, 18, 11, 1, 8],
 409,
 290)

In [260]:
vns(graph, random_solution, total_edge_length, 500, change_func=make_change_swap, shaking_func=shaking_inverse, local_search_func = local_search_with_permutation, k_min=3, k_max=6, move_prob=0.4)

([13, 7, 16, 17, 12, 3, 4, 6, 10, 14, 9, 19, 5, 0, 11, 18, 2, 1, 8, 15],
 430,
 217)

In [261]:
vns(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse, shaking_func=shaking_swap, local_search_func = local_search_with_permutation, k_min=1, k_max=3, move_prob=0.4)

([8, 1, 11, 2, 18, 16, 9, 17, 12, 6, 4, 7, 13, 14, 3, 5, 19, 0, 10, 15],
 414,
 146)

In [262]:
vns(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse, shaking_func=shaking_inverse, local_search_func = local_search_with_permutation, k_min=3, k_max=6, move_prob=0.4)

([15, 10, 0, 19, 5, 3, 14, 12, 7, 13, 6, 17, 4, 9, 18, 11, 16, 2, 1, 8],
 409,
 460)

In [263]:
vns(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble, shaking_func=shaking_swap, local_search_func = local_search_with_permutation, k_min=1, k_max=3, move_prob=0.4)

([15, 5, 0, 10, 19, 3, 14, 12, 18, 11, 9, 6, 4, 7, 17, 13, 2, 16, 1, 8],
 417,
 460)

In [264]:
vns(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble, shaking_func=shaking_inverse, local_search_func = local_search_with_permutation, k_min=3, k_max=6, move_prob=0.4)

([15, 7, 13, 10, 0, 4, 6, 3, 17, 16, 12, 5, 14, 9, 19, 2, 18, 11, 1, 8],
 407,
 321)